In [1]:
!pip install flask

In [2]:
!pip install scikit-learn

In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Считываем все файлы с диалогами из датасета

In [4]:
import pandas as pd
import glob

# Чтение всех CSV-файлов с диалогами
all_files = glob.glob("/content/drive/MyDrive/MFTI/NLP/HouseMD/*.csv")
df_list = [pd.read_csv(f, encoding='utf-8', encoding_errors='replace') for f in all_files]
df = pd.concat(df_list, ignore_index=True)

print(df.columns)
print(df.head(5))
print(f"Total lines: {len(df)}")


Index(['name', 'line'], dtype='object')
      name                                               line
0    House                              Pericardial effusion.
1     Taub   Which wasn't there last night when we did the...
2  Foreman   She's getting worse, and there's no sign of a...
3    Chase                                 Or maybe a cancer.
4  Foreman   We could use your opinion, House, or at least...
Total lines: 75312


Далее работаем только с репликами Грегори Хауса

In [5]:
house_lines = df[df['name'] == 'House'].copy()
print(house_lines.shape)
print(house_lines['line'].iloc[:5])

(20908, 2)
0                                 Pericardial effusion.
5      (taking off his glasses and looking up at the...
7      She was gonna kill herself. This is the final...
9          Yeah. I know why she wanted to kill herself.
10     His mentor, Helen Rutherford, has contracted ...
Name: line, dtype: object


Проводим предподготовку текста. Приводим текст к нижниму регистру, удаляем все кроме букв, токинизируем и удалем стоп слова по шаблону ntlk. Лемматизация не првела к удовлетворительному результату. В опытах со стеммингом результат приемлемый.

In [38]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Скачиваем необходимые данные NLTK (выполняется один раз)
nltk.download('punkt', force=True)
nltk.download('wordnet', force=True)
nltk.download('stopwords', force=True)
nltk.download('punkt_tab')
stemmer = PorterStemmer()

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text: str) -> str:
    """
    Функция для предобработки текста:
    - приведение к нижнему регистру,
    - удаление символов, отличных от букв,
    - токенизация,
    - удаление стоп-слов,
    - лемматизация.
    - Стемминг
    """
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление всего, кроме букв и пробелов
    text = re.sub(r'[^a-z\s]', '', text)
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stop_words]
    # Лемматизация
    # tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Стемминг
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    # Объединение обратно в строку
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [39]:
# house_lines['line'] – список исходных реплик
house_lines['Dialogue_processed'] = house_lines['line'].apply(preprocess_text)

Удаляем редки слова. Все слова которые встречаются резе чем 3х раз удаляются из корпуса

In [40]:
from collections import Counter

# Подсчет встречаемости слов в корпусе
word_counts = Counter(' '.join(house_lines['Dialogue_processed']).split())

# Убираем редкие слова (например, те, что встречаются <3 раз)
def remove_rare_words(text):
    return ' '.join([word for word in text.split() if word_counts[word] > 2])

house_lines['Dialogue_processed'] = house_lines['Dialogue_processed'].apply(remove_rare_words)


Проводим векторизацию текста с помощью TF-IDF

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))
# Обучаем векторизатор на всех репликах Доктора Dialogue_processed
tfidf_matrix = vectorizer.fit_transform(house_lines['Dialogue_processed'])
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

TF-IDF matrix shape: (20908, 109498)


Переводим запрос TF-IDF вектор и вычисляем косинусовое сходство вектора запроса и всеми векторами реплик Грегори Хауса

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

def find_best_reply(query_en: str) -> str:
    processed_query = preprocess_text(query_en)
    # Преобразуем запрос в TF-IDF
    query_vec = vectorizer.transform([processed_query])
    # Считаем косинусное сходство с каждой репликой (результат - массив значений)
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    # Индекс наиболее похожей реплики
    best_index = similarities.argmax()
    best_reply_en = house_lines['line'].iloc[best_index]
    return best_reply_en

# Пример поиска ответа на английский запрос
sample_query = "who lies?"  # пример запроса пользователя (англ.)
best_en = find_best_reply(sample_query)
print("Best match (EN):", best_en)


Best match (EN):  Everybody lies.


Используем SentenceTransformer для эмбедингов предложений

In [43]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(house_lines['Dialogue_processed'].tolist(), convert_to_tensor=True)

Проведем косинусовое сравнение семантически преобразованных эмбедингов запроса и реплик Грегори

In [48]:
def find_best_reply_semantic(query_en: str) -> str:
    processed_query = preprocess_text(query_en)
    query_emb = model.encode(processed_query, convert_to_tensor=True)
    # Считаем косинусные схожести и находим индекс максимального
    cos_scores = util.cos_sim(query_emb, embeddings)[0]
    best_index = int(cos_scores.argmax())
    return house_lines['line'].iloc[best_index]

# Пример семантического поиска
sample_query = "who lies?"  # пример запроса пользователя (англ.)  who lies?
best_en_sem = find_best_reply_semantic(sample_query)
print("Best semantic match (EN):", best_en_sem)
best_en = find_best_reply(sample_query)
print("Best match (EN):", best_en)

Best semantic match (EN):  You lied?
Best match (EN):  Everybody lies.


Вывод. Простое использование сравнений по косинусу векторов TF-IDF выглядит приемлемие чем использование предобученной модели и сравнение косинусов эмбедингов